# WRC Scraper  2020


*TO DO  - add in database elements. Schema will be in `wrcResults2020.sql`*

*Add in `asyncio` scheduler elements.*

In [1]:
from WRC_2020_scraper import *

In [2]:
class SeasonBase:
    """Base class for things to do with seasons."""
    def __init__(self, season_external_id=None, autoseed=False):
        self.season_external_id = season_external_id or None
        if not self.season_external_id and autoseed:
            self._check_season_external_id()
            
    def _check_season_external_id(self):
        """Check that season_external_id exists and if not, get one."""
        if not hasattr(self,'season_external_id') or not self.season_external_id:
            #Get current one from active rally
            #It's also available from current_season_events
            event, days, channels = getActiveRallyBase()
            self.event, self.days, self.channels = event, days, channels
            #The returned np.int64 is not JSON serialisable
            self.season_external_id = int(event.loc[0,'season.externalId'])

# TO DO
class Championship(SeasonBase):       
    """Class for championship."""
    def __init__(self ):
        SeasonBase.__init__(self)


In [3]:
SeasonBase(autoseed=True).season_external_id

6

In [4]:
# TO DO - define a class for each table
import warnings


class WRCRally_sdb:
    """Base class for things with an sdbRallyId.
       Can also help find an active sdbRallyId"""
    def __init__(self, sdbRallyId=None, live=False,
                 autoseed=False, nowarn=True,):
        if not nowarn and not sdbRallyId:
            warnings.warn("sdbRallyId should really be set...")
        
        self.sdbRallyId = sdbRallyId or None
        
        if autoseed:
            self._checkRallyId(sdbRallyId)
    
    def _checkRallyId(self, sdbRallyId=None):
        """Return a rally ID or lookup active one."""
        sdbRallyId = int(sdbRallyId) or self.sdbRallyId
        if not hasattr(self, 'sdbRallyId') or not self.sdbRallyId:
            self.activerally = WRCActiveRally()
            self.sdbRallyId = self.activerally.sdbRallyId
            self.name = self.activerally.name
        return self.sdbRallyId

In [5]:
class WRCActiveRally(WRCRally_sdb):
    """Class for the active rally."""
    def __init__(self, live=False ):
        WRCRally_sdb.__init__(self, live=live, nowarn=True)

        self.live = live
        self.fetchData()
        
    def fetchData(self):
        event, days, channels = getActiveRally()
        self.event, self.days, self.channels = event, days, channels

        #np.int64 is not JSON serialisable
        self.sdbRallyId = int(event.loc[0,'id'])

        self.name = event.loc[0,'name']

In [6]:
WRCActiveRally().sdbRallyId

100

In [7]:
zz = WRCRally_sdb(autoseed=True)
print(zz.sdbRallyId)

100


We use the `.fetchData()` method so as to ry not to be greedy. This way, we can define a class and start to work towards only grabbling the data if we need it.

In [8]:
class WRCRetirements(WRCRally_sdb):
    """Callable class for retirements"""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):
        """Initialise retirements class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId,
                              live=live, autoseed=autoseed)
            
        self.retirements=None
        
        if self.sdbRallyId:
            self.fetchData(self.sdbRallyId)
        
    def fetchData(self, sdbRallyId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        self.retirements = getRetirements(self.sdbRallyId)
    
    def __call__(self):
        return self.retirements

In [9]:
zz=WRCRetirements(autoseed=True)
zz.retirements.head(3)

,retirementId,controlId,entryId,reason,retirementDateTime,retirementDateTimeLocal,status
0,1475,6591,20710,OFF ROAD,2020-01-23T20:40:00Z,0001-01-01T00:00:00+00:00,Temporary
1,1476,6588,20687,MECHANICAL,2020-01-23T21:57:00Z,0001-01-01T00:00:00+00:00,Temporary
2,1477,6591,20750,OFF ROAD,2020-01-23T22:51:00Z,0001-01-01T00:00:00+00:00,Permanent


In [10]:
class WRCPenalties(WRCRally_sdb):
    """Callable class for penalties."""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):
        """Initialise penalties class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId,
                              live=live, autoseed=autoseed)
            
        self.penalties=None
        
        if self.sdbRallyId:
            self.fetchData(self.sdbRallyId)

    
    def fetchData(self, sdbRallyId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        self.penalties = getPenalties(self.sdbRallyId)
    
    def __call__(self):
        return self.penalties

In [11]:
zz=WRCPenalties(autoseed=True)
zz.penalties.head(3)

,penaltyId,controlId,entryId,penaltyDurationMs,penaltyDuration,reason
0,725,6592,20730,10000,PT10S,FALSE START
1,726,6592,20753,10000,PT10S,FALSE START
2,727,6590,20760,10000,PT10S,1 MIN LATE


In [12]:
zz.name

'Rallye Monte Carlo'

In [13]:
zz().head(2)

,penaltyId,controlId,entryId,penaltyDurationMs,penaltyDuration,reason
0,725,6592,20730,10000,PT10S,FALSE START
1,726,6592,20753,10000,PT10S,FALSE START


In [14]:
class WRCItinerary(WRCRally_sdb):
    """Class for WRC2020 Itinerary."""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):
        """Initialise itinerary class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId,
                              live=live, autoseed=autoseed)
        
        self.itinerary=None
        self.legs=None
        self.sections=None
        self.controls=None
        self.stages=None
        
        if self.sdbRallyId:
            self.fetchData(sdbRallyId)
        
    def fetchData(self, sdbRallyId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        
        itinerary, legs, sections, controls, stages = getItinerary(self.sdbRallyId)
        self.itinerary, self.legs, self.sections, self.controls, self.stages = itinerary, legs, sections, controls, stages
        

In [15]:
print(WRCItinerary(autoseed=True).sdbRallyId)

100


In [16]:
WRCItinerary(sdbRallyId=100).legs

,itineraryLegId,itineraryId,startListId,name,legDate,order,status
0,273,240,451,Thursday 23rd January,2020-01-23,1,Completed
1,272,240,452,Friday 24th January,2020-01-24,2,Completed
2,275,240,454,Saturday 25th January,2020-01-25,3,Completed
3,274,240,456,Sunday 26th January,2020-01-25,4,Completed


In [17]:
class WRCStartlist():
    """Class for WRC2020 Startlist table."""
    def __init__(self, startListId=None, autoseed=True):
        self.startListId = startListId or None
        
        if not self.startListId:
            warnings.warn("startListId should really be set..")
        
        if self.startListId or autoseed:
            self.fetchData(self.startListId)
    
    def _checkStartListId(self, startListId=None):
        """Return a startlistId or look one up."""
        self.startListId = startListId or self.startListId
        if not self.startListId:
            if not hasattr(self, 'itinerary') or not self.itinerary:
                self.itinerary = WRCItinerary(autoseed=True)
                self.sdbRallyId = self.itinerary.sdbRallyId
            self.startListId = int(self.itinerary.legs.loc[0,'startListId'])
        return self.startListId
        
        
    def fetchData(self, startListId=None):
        self._checkStartListId(startListId)
        startList,startListItems = getStartlist(self.startListId)
        self.startList, self.startListItems = startList,startListItems

In [18]:
WRCStartlist(autoseed=True).startList

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: startListId should really be set..
  import sys


,startListId,eventId,publishedStatus,name
0,451,124,Published,Thursday


In [19]:
class WRCCars(WRCRally_sdb):
    """Class for WRC2020 Cars table."""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):  
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId,
                              live=live, autoseed=autoseed)
        
        self.cars=None
        self.classes=None
        
        if self.sdbRallyId:
            self.fetchData(sdbRallyId)
            
    def fetchData(self, sdbRallyId=None):
        self._checkRallyId(sdbRallyId)
        cars, classes = getCars(self.sdbRallyId)
        self.cars, self.classes = cars, classes

In [20]:
WRCCars(autoseed=True)

In [24]:
class WRCRally(WRCRally_sdb):
    """Class for WRC2020 Rally table."""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):  
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId,
                              live=live, autoseed=autoseed)
        
        self.rally=None
        self.eligibilities=None
        self.groups=None
        
        if self.sdbRallyId:
            self.fetchData(sdbRallyId)
            
    def fetchData(self, sdbRallyId=None):
        self._checkRallyId(sdbRallyId)
        cars, classes = getCars(sdbRallyId)
        rally, eligibilities, groups = getRally(self.sdbRallyId)
        self.rally, self.eligibilities = rally, eligibilities

In [27]:
zz=WRCRally(autoseed=True)
zz.eligibilities

,0
0,M
1,None
2,WRC2
3,WRC3
4,RGT


In [21]:
# TO DO
#splitPoints, entrySplitPointTimes, splitPointTimes = getSplitTimes(sdbRallyId,stageId)
#stagetimes = getStageTimes(sdbRallyId,stageId)
#stagewinners = getStagewinners(sdbRallyId)
#championship = getChampionship()
#championship = getChampionshipStandings()

In [73]:
#paths=[]
#parts=attr.split('.')
#for i, _ in enumerate(parts):
#    if i:
#        paths.append('.'.join(parts[:-i]))
#paths.reverse()

def _checkattr(obj,attr):
    """Check an object exists and is set to a non-null value."""
    
    #TO DO  - support attributes done a path, checking each step in turn
    
    if hasattr(obj,attr):
        if isinstance(obj.attr, pd.DataFrame):
            return not obj.attr.empty
        elif obj.attr:
            return True
        
    return False

In [91]:
class WRCRally_stage(WRCRally_sdb):
    """Base class for things with a stageId.
       Can also help find a stageId list for a given rally."""
    def __init__(self, sdbRallyId=None, stageId=None, live=False,
                 autoseed=False, nowarn=True,):
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId,
                              live=live, autoseed=autoseed, nowarn=False)
        
        if not nowarn and not sdbRallyId:
            warnings.warn("sdbRallyId should really be set...")
        if not nowarn and not stageId:
            warnings.warn("stageId should really be set...")
        
        self.sdbRallyId = sdbRallyId or None
        self.stageId = stageId or None
        
        if autoseed:
            self._rallyStages(self.sdbRallyId, self.stageId)
    
    def _rallyStages(self, sdbRallyId=None, stageId=None):
        """Return a stageId list or lookup list of active rally."""
        stageId = stageId or self.stageId
        
        #Have we got an sdbRallyId?
        if not hasattr(self, 'sdbRallyId') or not self.sdbRallyId:
            self.activerally = WRCActiveRally()
            self.sdbRallyId = self.activerally.sdbRallyId
            self.name = self.activerally.name
        
        #The stages are in the stages return value from the itinerary
        #itinerary, legs, sections, controls, stages = getItinerary(sdbRallyId)
        if not hasattr(self, 'itinerary') or not _checkattr(self,'itinerary.stages'):
            self.itinerary = WRCItinerary(self.sdbRallyId, autoseed=True)
        
        _ccols=['code']+(list(set(self.itinerary.controls.columns) - set(self.itinerary.stages.columns)))
        return (self.sdbRallyId, self.itinerary.stages.merge(self.itinerary.controls[_ccols], on='code'))

In [92]:
zz=WRCRally_stage(autoseed=True)
zz.itinerary.stages # stages / controls
zz._rallyStages()[1]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: sdbRallyId should really be set...
  # This is added back by InteractiveShellApp.init_path()


,stageId,eventId,number,name,distance,status,stageType,timingPrecision,locked,code,type,firstCarDueDateTime,controlPenalties,roundingPolicy,targetDuration,controlId,firstCarDueDateTimeLocal,location,targetDurationMs
0,1528,124,1,Malijai - Puimichel (Live TV),17.47,Interrupted,SpecialStage,Tenth,True,SS1,StageStart,2020-01-23T19:38:00,None,RoundToClosestMinute,00:03:00,6592,2020-01-23T20:38:00+01:00,Malijai - Puimichel (Live TV),180000.0
1,1538,124,2,Bayons - Bréziers,25.49,Completed,SpecialStage,Tenth,True,SS2,StageStart,2020-01-23T21:26:00,None,RoundToClosestMinute,00:03:00,6589,2020-01-23T22:26:00+01:00,Bayons - Bréziers,180000.0
2,1533,124,3,Curbans - Venterol 1,20.02,Completed,SpecialStage,Tenth,True,SS3,StageStart,2020-01-24T07:36:00,None,RoundToClosestMinute,00:03:00,6586,2020-01-24T08:36:00+01:00,Curbans - Venterol 1,180000.0
3,1534,124,4,Saint-Clément - Freissinières 1,20.68,Completed,SpecialStage,Tenth,True,SS4,StageStart,2020-01-24T08:56:00,None,RoundToClosestMinute,00:03:00,6583,2020-01-24T09:56:00+01:00,Saint-Clément - Freissinières 1,180000.0
4,1535,124,5,Avançon - Notre-Dame-du-Laus 1,20.59,Completed,SpecialStage,Tenth,True,SS5,StageStart,2020-01-24T10:21:00,None,RoundToClosestMinute,00:03:00,6547,2020-01-24T11:21:00+01:00,Avançon - Notre-Dame-du-Laus 1,180000.0
5,1539,124,6,Curbans - Venterol 2,20.02,Completed,SpecialStage,Tenth,True,SS6,StageStart,2020-01-24T12:54:00,None,RoundToClosestMinute,00:03:00,6548,2020-01-24T13:54:00+01:00,Curbans - Venterol 2,180000.0
6,1537,124,7,Saint-Clément - Freissinières 2,20.68,Completed,SpecialStage,Tenth,True,SS7,StageStart,2020-01-24T14:14:00,None,RoundToClosestMinute,00:03:00,6551,2020-01-24T15:14:00+01:00,Saint-Clément - Freissinières 2,180000.0
7,1536,124,8,Avançon - Notre-Dame-du-Laus 2,20.59,Completed,SpecialStage,Tenth,True,SS8,StageStart,2020-01-24T15:39:00,None,RoundToClosestMinute,00:03:00,6554,2020-01-24T16:39:00+01:00,Avançon - Notre-Dame-du-Laus 2,180000.0
8,1532,124,9,St-Léger-les-Mélèzes - La Bâtie-Neuve 1,16.87,Completed,SpecialStage,Tenth,True,SS9,StageStart,2020-01-25T08:38:00,None,RoundToClosestMinute,00:03:00,6557,2020-01-25T09:38:00+01:00,St-Léger-les-Mélèzes - La Bâtie-Neuve 1,180000.0
9,1531,124,10,La Bréole - Selonnet 1,20.73,Completed,SpecialStage,Tenth,True,SS10,StageStart,2020-01-25T09:56:00,None,RoundToClosestMinute,00:03:00,6560,2020-01-25T10:56:00+01:00,La Bréole - Selonnet 1,180000.0


In [43]:
class WRCSplitTimes(WRCRally_sdb):
    """Class for WRC2020 SplitTimes table."""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):  
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId,
                              live=live, autoseed=autoseed)
        
        self.rally=None
        self.eligibilities=None
        self.groups=None
        
        if self.sdbRallyId:
            self.fetchData(sdbRallyId)
            
    def fetchData(self, sdbRallyId=None):
        self._checkRallyId(sdbRallyId)
        cars, classes = getCars(sdbRallyId)
        rally, eligibilities, groups = getRally(self.sdbRallyId)
        self.rally, self.eligibilities = rally, eligibilities

In the following class:

    - `fetchX` will grab a copy from the WRC API;
    - `getX` will try to use a copy on the event object, and if it doesnlt exist, try to fetch it before returning it;

In [31]:
#This class will contain everything about a single rally
class WRCEvent(WRCRally_sdb):
    """Class for a rally - stuff where sdbRallyId is required."""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False, slurp=False):
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId, live=live,
                             autoseed=autoseed)
        
        self.live = live
        self.itinerary = None
        self.startListId = None
        self.activerally = None
    
        if slurp:
            self.rallyslurper()
    
    def _setItinerary(self):
        """Set itinerary.
           If rally not known, use active rally.
           Also set a default startListId."""
        
        self.itinerary = WRCItinerary(self.sdbRallyId, autoseed=True)
        
        _i = self.itinerary
        #If necessary, set a default sdbRallyId
        if not self.sdbRallyId:
             self.sdbRallyId = _i.sdbRallyId
                
        #Set a default startListId value if required
        if not self.startListId and _i and hasattr(_i,'legs') and not _i.legs.empty:
            self.startListId = int(_i.legs.loc[0,'startListId'])

        #return (_i.itinerary, _i.legs, _i.sections, _i.controls, _i.stages)
        
    def _checkItinerary(self):
        """Check itinerary.
           If rally not known, use active rally.
           Also set a default startListId."""
        
        _itinerary_items = ['itinerary', 'legs', 'sections', 'controls', 'stages']
        if not any([hasattr(self, i) for i in _itinerary_items]):
            _setItinerary(self)
        return 
            
    def fetchItinerary(self):
        """Get itinerary.
           If rally not known, use active rally.
           Also set a default startListId."""
        self._setItinerary(self)
        return (self.itinerary, self.legs, self.sections, self.controls, self.stages)
 
    def fetchCars(self):
        """Get cars for a rally.
           If no rally provided, use current one."""
        
        _c = self.cars = WRCCars(self.sdbRallyId)
        
        return (_c.cars, _c.classes)
       
        
    def fetchStartlist(self, startListId=None):
        """Get startlist.
           If no startListId provided, try to find a default."""
        
        _s = self.startlist = WRCStartlist(self.startListId)
      
        return (_s.startList, _s.startListItems)
    
    
    def fetchPenalties(self):
        """Get penalties."""
        
        self.penalties = WRCPenalties(self.sdbRallyId)()
        return self.penalties
     
        
    def fetchRetirements(self):
        """Get retirements."""
        
        self.retirements = WRCRetirements(self.sdbRallyId)()
        return self.retirements
     
    
    def rallyslurper(self):
        """Grab everything..."""
        self.getItinerary()
        self.getCars()
        self.getStartlist()
        self.getPenalties()
        self.getRetirements()
    

In [23]:
# NEXT TO DO - active rally class

In [24]:
#zz = WRCEvent(slurp=True)

In [25]:
zz = WRCEvent(autoseed=True)
zz.getPenalties()

,penaltyId,controlId,entryId,penaltyDurationMs,penaltyDuration,reason
0,725,6592,20730,10000,PT10S,FALSE START
1,726,6592,20753,10000,PT10S,FALSE START
2,727,6590,20760,10000,PT10S,1 MIN LATE
3,728,6590,20764,50000,PT50S,5 MINS LATE
4,729,6590,20769,10000,PT10S,1 MIN LATE


In [32]:
# TO DO - need a more gernal season events class?
# If, that is, we can we look up arbtrary season events...
class WRCCurrentSeasonEvents:
    """Class for Season events."""
    def __init__(self ):
        self.current_season_events, self.eventdays, self.eventchannel = getCurrentSeasonEvents()


In [39]:
#This class needs renaming...
#What does it actually represent? An event? A live event? A set of events?
class WRC2020(WRCEvent):
    """Class for WRC data scrape using 2020 API."""

    def __init__(self, sdbRallyId=None, live=False):
        WRCEvent.__init__(self, sdbRallyId, live)
        
        self.live = live
        self.currentseasonevents = None
        
    
    def fetchCurrentSeasonEvents(self):
        """Fetch current season events."""
        self.currentseasonevents = WRCCurrentSeasonEvents()
        
    def getCurrentSeasonEvents(self):
        """Get Current  = season events."""
        _current_season_sevents_attrs = ['currentseasonevents', 'current_season_events',
                                         'eventdays', 'eventchannel' ]
        if not any([hasattr(self,a) for a in _current_season_sevents_attrs]) or not self.currentseasonevents:
            _cse = self.currentseasonevents = WRCCurrentSeasonEvents()
        else:
            _cse = self.currentseasonevents
        return (_cse.current_season_events, _cse.eventdays, _cse.eventchannel)


    
    
    
    

In [40]:
wrc=WRC2020()
wrc.getCurrentSeasonEvents()
wrc.currentseasonevents.current_season_events

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,images.format16x9.320x180,images.format16x9.160x90,...,winner.birthDate,winner.birthPlace,winner.debutDate,winner.debutPlace,winner.website,winner.driverImageFormats,winner.externalId,winner.page,winner,seasonYear
0,100,Rallye Monte Carlo,153,124,1,True,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,1988-06-16,Belgium,2009-01-12,Rally de Portugal,https://www.thierryneuville.com/,"[{'id': 6, 'title': 'Format 16:9', 'imageForma...",762.0,NaN,NaN,2020
1,102,Rally Sweden,154,125,2,False,False,True,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
2,107,Rally Guanajuato Mexico,155,126,-6,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
3,114,Rally Argentina,156,127,-3,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
4,116,Rally de Portugal,157,128,1,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
5,126,Rally Italia Sardegna,158,129,2,False,False,True,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
6,137,Safari Rally Kenya,159,130,3,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
7,148,Neste Rally Finland,160,131,3,False,False,True,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
8,157,Rally New Zealand,161,132,12,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
9,158,Rally Turkey,162,133,3,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020


In [456]:
wrc.getStartlist()

(   startListId  eventId publishedStatus      name
 0          451      124       Published  Thursday,
     startListItemId  startListId  entryId         startDateTime  \
 0             20891          451    20745  2020-01-23T17:37:00Z   
 1             20892          451    20743  2020-01-23T17:36:00Z   
 2             20893          451    20722  2020-01-23T17:35:00Z   
 3             20894          451    20741  2020-01-23T17:34:00Z   
 4             20895          451    20740  2020-01-23T17:33:00Z   
 ..              ...          ...      ...                   ...   
 83            20974          451    20702  2020-01-23T16:55:00Z   
 84            20975          451    20703  2020-01-23T16:54:00Z   
 85            20976          451    20707  2020-01-23T16:53:00Z   
 86            20977          451    20728  2020-01-23T17:15:00Z   
 87            20978          451    20683  2020-01-23T16:00:00Z   
 
            startDateTimeLocal  order  
 0   2020-01-23T18:37:00+01:00     64  

In [325]:
wrc.itinerary.sections

,itinerarySectionId,itineraryLegId,order,name
0,637,273,1,Section 1
1,638,272,2,Section 2
2,639,272,3,Section 3
3,640,275,4,Section 4
4,641,275,5,Section 5
5,642,274,6,Section 6
6,643,274,7,Section 7


In [280]:
itinerary, legs, sections, controls, stages = wrc.getItinerary()
startList,startListItems = getStartlist(startListId)
cars, classes = getCars(sdbRallyId)
#rally, eligibilities, groups = getRally(sdbRallyId)
#overall = getOverall(sdbRallyId, stageId)
#splitPoints, entrySplitPointTimes, splitPointTimes = getSplitTimes(sdbRallyId,stageId)
#stagetimes = getStageTimes(sdbRallyId,stageId)
#stagewinners = getStagewinners(sdbRallyId)
#penalties = getPenalties(sdbRallyId)
#retirements = getRetirements(sdbRallyId)
#championship = getChampionship()
#championship = getChampionshipStandingsLive()

In [236]:
current_season_events, eventdays, eventchannel = wrc.getCurrentSeasonEvents()

In [242]:
event, days, channels = wrc.getActiveRally()


In [238]:
event

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,winner,images.format16x9.320x180,...,status.id,status.name,pageInfo.id,pageInfo.title,pageInfo.feTitle,pageInfo.url,pageResult.id,pageResult.title,pageResult.feTitle,pageResult.url
0,100,Rallye Monte Carlo,153,124,1,True,False,False,None,https://www.wrc.com/images/redaktion/Web-2020/...,...,21,Live Event,38227,Rallye-Monte-Carlo,Rallye Monte-Carlo,/en/championship/calendar/wrc/rallye-monte-car...,89623,Rally Monte Carlo,Rally Monte Carlo,/en/results/rally-results/rally-monte-carlo/re...


In [243]:
wrc.sdbRallyId

100